<a href="https://colab.research.google.com/github/KalyaniVijayPakhale/QnA-with-LLM-embeddings/blob/main/Research_POC1_context_citation_for_single_document.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain --upgrade
!pip install openai
!pip install chromadb
!pip install pymupdf
!pip install vectorstore
!pip install tiktoken
!pip install python-dotenv

In [2]:
pip install PyMuPDF

In [3]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
import os
from langchain.docstore.document import Document
load_dotenv()
import fitz
import vectorstore
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
#import pinecone
from langchain.docstore.document import Document
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import create_citation_fuzzy_match_chain
from langchain.chat_models import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.agents import AgentExecutor, Tool,initialize_agent
from langchain.agents.types import AgentType

In [4]:
# Load the data
def pdf_to_text(pdf_path, output_text_path):
    doc = fitz.open(file_path)
    text = ""

    for page_num in range(doc.page_count):
        page = doc[page_num]
        text += page.get_text()

    doc.close()

    with open(output_text_path, "w", encoding="utf-8") as text_file:
        text_file.write(text)

file_path = "/content/drive/MyDrive/Information Extraction/Research POC/citation/Doc_1.PDF"
text_file_path = "/content/drive/MyDrive/Information Extraction/Research POC/citation/Doc_1.txt"

pdf_to_text(file_path, text_file_path)

In [8]:
import fitz

def get_pdf_metadata(file_path):
    try:
        pdf_document = fitz.open(file_path)

        # Get document metadata
        metadata = pdf_document.metadata
        # no_of_pages = len(doc)
        # for i in range(no_of_pages):
        #     page_text = doc[i].get_text()
        #     page_metadata = {"Source pdf": file_name, "Page no": i}
        #     page = Document(page_content=page_text, metadata=page_metadata)
        #     document.append(page)

        # Print basic metadata information
        print("Title:", metadata.get("title", "N/A"))
        # print("Author:", metadata.get("author", "N/A"))
        # print("Subject:", metadata.get("subject", "N/A"))
        # print("Creator:", metadata.get("creator", "N/A"))
        # print("Producer:", metadata.get("producer", "N/A"))
        print("Creation Date:", metadata.get("creationDate", "N/A"))
        print("Modification Date:", metadata.get("modDate", "N/A"))

        print("Number of Pages:", pdf_document.page_count)
        # print(document[0])




    except Exception as e:
        print(f"An error occurred: {e}")

file_path = "/content/drive/MyDrive/Information Extraction/Research POC/citation/Doc_1.PDF"
get_pdf_metadata(file_path)

Title: Basel III Capital Regulations
Creation Date: D:20230411145032+05'30'
Modification Date: D:20230411152613+05'30'
Number of Pages: 9


In [9]:
# now we get all pdf metadata
loader = TextLoader(text_file_path)
data = loader.load()
data

[Document(page_content=' \nRBI/2023-24/14 \nDOR.SFG.REC.10/30.01.021/2023-24 \n \n \n \n                    April 11, 2023 \n  \n \nAll Scheduled Commercial Banks including Small Finance Banks  \n(excluding Regional Rural Banks, Local Area Banks and Payments Banks) \nAll Deposit taking Non-Banking Finance Companies (NBFCs) including Housing Finance \nCompanies (HFCs) \n \n \nMadam/Dear Sir, \nFramework for acceptance of Green Deposits \nClimate change has been recognised as one of the most critical challenges faced by the global \nsociety and economy in the 21st century. The financial sector can play a pivotal role in \nmobilizing resources and their allocation thereof in green activities/projects. Green finance is \nalso progressively gaining traction in India. \n2. Deposits constitute a major source for mobilizing of funds by the Regulated Entities (REs). \nIt is seen that some REs are already offering green deposits for financing green activities and \nprojects. Taking this forward 

In [10]:
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[0].page_content)} characters in your sample document')
print (f'Here is a sample: {data[0].page_content[:200]}')

You have 1 document(s) in your data
There are 15784 characters in your sample document
Here is a sample:  
RBI/2023-24/14 
DOR.SFG.REC.10/30.01.021/2023-24 
 
 
 
                    April 11, 2023 
  
 
All Scheduled Commercial Banks including Small Finance Banks  
(excluding Regional Rural Banks, Local


In [11]:
# We'll split our data into chunks around 500 characters each with a 50 character overlap. These are relatively small.

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(data)
# Let's see how many small chunks we have
print (f'Now you have {len(texts)} documents')

Now you have 36 documents


In [12]:
OPENAI_API_KEY= ''
os.environ["OPENAI_API_KEY"] = ''
embeddings = OpenAIEmbeddings()

In [13]:
vectorstore = Chroma.from_documents(texts, embeddings)
query = "What is green deposits?" #"What is aka corp dev group is about?"
docs = vectorstore.similarity_search(query)

In [14]:
for doc in docs:
    print (f"{doc.page_content}\n")

allocation of green deposit 
proceeds pending their 
allocation to the eligible green 
activities/projects 
 
 
 
 
* This shall contain the cumulative amount since the RE started offering green deposits. For example, 
if a bank has commenced raising green deposits from June 1, 2023, then the annual financial statement 
for the period ending March 31, 2025 would contain particulars of deposits raised and allocated from 
June 1, 2023 till March 31, 2025.

2. 
Definitions 
In these guidelines, unless the context states otherwise, the terms herein shall bear the 
meaning assigned to them below: 
(a) "green activities/projects” means the activities/projects meeting the requirements 
prescribed in paragraph 7 of these guidelines; 
(b) “green deposit” means an interest-bearing deposit, received by the RE for a fixed period 
and the proceeds of which are earmarked for being allocated towards green finance;

6Feedstock will primarily include: Sewage, manure, wastewater, bagasse, biomass, wood 

In [17]:
llm = ChatOpenAI(temperature=0, model="gpt4")
prompt=r"""Generate a descriptive answer from multiple references and give citation contaxt.
            If you are not able to generate the answer, just say not able to generate answer for the question from the given context.
            Consider the context that was given and do not think out of the context to generate the answer.
            {context}
            Question: {query}
            Answer:
            """
llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = create_citation_fuzzy_match_chain(llm)
query = "What is green deposits?"  #"What is aka corp dev group is about?"
def gen_ans(query):
  retieval = chain.run(context=docs,question=query,return_source_documents=True)#, Title=title, page_no=page_number)
  return(retieval)

#qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())
gen_ans(query)

QuestionAnswer(question='What is green deposits?', answer=[FactWithEvidence(fact='A green deposit means an interest-bearing deposit, received by the RE for a fixed period and the proceeds of which are earmarked for being allocated towards green finance.', substring_quote=['“green deposit” means an interest-bearing deposit, received by the RE for a fixed period and the proceeds of which are earmarked for being allocated towards green finance;'])])

In [18]:
retieval = chain.run(context=docs,question=query,return_source_documents=True)#, Title=title, page_no=page_number)

def highlight(text, span):
    return (
        "..."
        + text[span[0] - 20 : span[0]]
        + "*"
        + "\033[91m"
        + text[span[0] : span[1]]
        + "\033[0m"
        + "*"
        + text[span[1] : span[1] + 20]
        + "..."
    )

for fact in retieval.answer:
    print("Statement:", fact.fact)

Statement: A green deposit means an interest-bearing deposit, received by the RE for a fixed period and the proceeds of which are earmarked for being allocated towards green finance.


In [19]:
gen_ans(query)

def extract_target_sentences_from_retrieval(retrieval):
    question = retrieval.question
    facts_with_evidence = retrieval.answer

    all_target_sentences = []

    if facts_with_evidence:
        for fact in facts_with_evidence:
            if fact.substring_quote:
                all_target_sentences.extend(fact.substring_quote)
    target_sentences = ' '.join(all_target_sentences)

    return target_sentences

target_sentences = extract_target_sentences_from_retrieval(retieval)
if target_sentences:
    print(f"Target Context: {target_sentences}")
else:
    print("No substring_quotes found in the retrieval output.")
def search_all_pages(document, target_context):
    results = []

    for page_number in range(document.page_count):
        page = document[page_number]

        search_results = page.search_for(target_context)

        # If there are search results on the current page, add them to the overall results
        if search_results:
            for result in search_results:
                results.append({
                    "Page Number": page_number + 1,  # Page numbers start from 1
                    "Rect": result,
                    "Text": page.get_text("text", clip=result)
                })

    return results


doc = fitz.open(file_path)

search_results = search_all_pages(doc, target_sentences)

for result in search_results:
    print(f"Page Number: {result['Page Number']}")
    print(f"Rect: {result['Rect']}")
    print(f"Text: {result['Text']}\n")

def extract_file_name(path):
  file_name = os.path.basename(path)
  return (print('File=',file_name))

extract_file_name(file_path)
get_pdf_metadata(file_path)

Target Context: “green deposit” means an interest-bearing deposit, received by the RE for a fixed period and the proceeds of which are earmarked for being allocated towards green finance;
Page Number: 2
Rect: Rect(85.08000183105469, 544.9201049804688, 198.85293579101562, 557.1868286132812)
Text: “green deposit” means 


Page Number: 2
Rect: Rect(199.33999633789062, 543.0564575195312, 520.472412109375, 558.1430053710938)
Text: an interest-bearing deposit, received by the RE for a fixed period 


Page Number: 2
Rect: Rect(85.08000183105469, 562.0164184570312, 495.4524230957031, 577.1029663085938)
Text: and the proceeds of which are earmarked for being allocated towards green finance;


File= Doc_1.PDF
Title: Basel III Capital Regulations
Creation Date: D:20230411145032+05'30'
Modification Date: D:20230411152613+05'30'
Number of Pages: 9
